# Analisis Sentimen Menggunakan OpenAI API

Notebook ini berisi langkah-langkah untuk melakukan analisis sentimen pada data teks menggunakan OpenAI API.

## 1. Setup Lingkungan dan Inisialisasi API

Bagian ini melakukan instalasi library yang diperlukan (`openai`, `pandas`, `google-colab`) dan menginisialisasi koneksi ke OpenAI API menggunakan kunci API yang disimpan di Colab Secrets.

In [85]:
# --- SETUP DAN INSTALASI ---
!pip uninstall -y openai
!pip install -q openai pandas google-colab

import os
import pandas as pd
from openai import OpenAI, APIError
from google.colab import userdata

# --- INISIALISASI API ---
try:
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI")
    OPENAI_MODEL = "gpt-4.1"  # ganti sesuai model kamu
    client = OpenAI()
    print(f"Inisialisasi OpenAI API berhasil dengan model: {OPENAI_MODEL}")
except Exception as e:
    print("❌ Gagal inisialisasi OpenAI client.")
    print("Pastikan kunci API di Colab Secrets bernama 'OPENAI' dan nilainya valid.")
    raise

Inisialisasi OpenAI API berhasil dengan model: gpt-4.1


## 2. Konfigurasi File dan Definisi Fungsi Analisis Sentimen

Bagian ini mendefinisikan nama file input dan output, serta kolom teks yang akan dianalisis. Juga mendefinisikan fungsi `analyze_sentiment_openai` yang bertanggung jawab untuk mengirim teks ke OpenAI API dan mengurai respons sentimennya.

In [86]:
# --- KONFIGURASI FILE DAN FUNGSI ANALISIS SENTIMEN ---
INPUT_FILE = "5-garudaindonesia_news_stem.csv"
OUTPUT_FILE = "6-garudaindonesia_news_stem_tag_new_dan_sentiment.csv"
COLUMN_TO_ANALYZE = "konten"

VALID_TAGS = [
    "Kinerja & Keuangan",
    "Operasional & Pelayanan",
    "Regulasi & Kebijakan",
    "Krisis & Kontroversi",
    "Industri & Pariwisata Nasional"
]

def analyze_sentiment_openai(text_to_analyze):
    if pd.isna(text_to_analyze) or not isinstance(text_to_analyze, str) or not text_to_analyze.strip():
        return {"sentiment": "Not Applicable", "tag_new": "Not Applicable"}

    system_instruction = (
    "Kamu adalah AHLI analisis sentimen yang sangat AKURAT. "
    "Untuk setiap teks, kamu harus mengeluarkan output dalam format JSON lengkap seperti contoh berikut:\n\n"
    'Contoh:\n'
    '{"sentiment": "Positive", "tag_new": "Operasional & Pelayanan"}\n'
    '{"sentiment": "Neutral", "tag_new": "Kinerja & Keuangan"}\n'
    '{"sentiment": "Negative", "tag_new": "Krisis & Kontroversi"}\n\n'
    "Selalu isi kedua field dengan nilai yang sesuai.\n"
    "Nilai sentiment hanya boleh: Positive, Neutral, atau Negative.\n"
    "Nilai tag hanya boleh salah satu dari lima kategori berikut:\n"
    "1. Kinerja & Keuangan\n"
    "2. Operasional & Pelayanan\n"
    "3. Regulasi & Kebijakan\n"
    "4. Krisis & Kontroversi\n"
    "5. Industri & Pariwisata Nasional\n\n"
    "Jangan kosongkan field apapun. Jangan tulis apapun di luar JSON."
)



    try:
        response = client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=[
                {"role": "system", "content": system_instruction},
                {"role": "user", "content": f"Teks: {text_to_analyze}"}
            ],
            max_completion_tokens=60
        )

        raw_output = response.choices[0].message.content.strip()

        import json
        try:
            parsed = json.loads(raw_output)
            sentiment_label = parsed.get("sentiment", "").strip()
            tag_label = parsed.get("tag_new", "").strip()
        except Exception:
            sentiment_label, tag_label = "Unclassified", "Operasional & Pelayanan"

        # --- validasi sentiment ---
        valid_labels = ["Positive", "Neutral", "Negative"]
        sentiment_label = next(
            (lbl for lbl in valid_labels if lbl.lower() == sentiment_label.lower()),
            next((lbl for lbl in valid_labels if lbl.lower() in sentiment_label.lower()), "Unclassified")
        )

        # --- validasi tag ---
        tag_label_clean = next((t for t in VALID_TAGS if t.lower() == tag_label.lower()), None)
        if not tag_label_clean:
            # fallback ke kategori paling umum
            tag_label_clean = "Operasional & Pelayanan"

        return {"sentiment": sentiment_label, "tag_new": tag_label_clean}

    except APIError as e:
        return {"sentiment": f"API Error: {e.status_code} - {e.message}", "tag_new": "Error"}
    except Exception as e:
        return {"sentiment": f"Processing Error: {e}", "tag_new": "Error"}


## 3. Uji Coba Fungsi Analisis Sentimen

Sebelum memproses seluruh dataset, bagian ini melakukan uji coba fungsi `analyze_sentiment_openai` dengan beberapa contoh teks. Hal ini penting untuk memverifikasi bahwa fungsi bekerja sesuai harapan dan tidak ada masalah dengan koneksi API atau parameter model.

In [87]:
# ----------------------------------------------------------------------
# --- UJI COBA FUNGSI ANALISIS SENTIMEN ---
# Bagian ini melakukan uji coba fungsi `analyze_sentiment_openai`
# dengan beberapa contoh teks untuk memastikan fungsi bekerja dengan benar
# sebelum diterapkan ke seluruh dataset.
# ----------------------------------------------------------------------
print("\n--- UJI Coba Fungsi Sentimen ---")

test_texts = [
    "Pelayanan di maskapai ini sangat memuaskan, saya akan terbang lagi!",
    "Berita tentang pergantian direktur utama ini tidak menimbulkan sentimen apapun.",
    "Pembatalan penerbangan tanpa pemberitahuan adalah pengalaman yang sangat mengecewakan.",
    "Seperti Labuan Bajo, Garuda mendukung pariwisata nasional. Meskipun begitu, Garuda jelek untuk pariwisata."
]

for i, text in enumerate(test_texts):
    print(f"Menguji Teks {i+1}...")
    test_sentiment = analyze_sentiment_openai(text)

    print(f"Teks: '{text[:50]}...'")
    print(f"Sentimen Hasil: **{test_sentiment}**")
    print("-" * 20)

    # Hentikan uji coba jika ada API Error
    if "API Error" in test_sentiment:
        print("\n❌ GAGAL UJI COBA. EKSEKUSI DATA DIHENTIKAN. Cek kembali parameter untuk model Anda atau kuota API Anda.")
        break

print("--- Uji Coba Selesai ---")


--- UJI Coba Fungsi Sentimen ---
Menguji Teks 1...
Teks: 'Pelayanan di maskapai ini sangat memuaskan, saya a...'
Sentimen Hasil: **{'sentiment': 'Positive', 'tag_new': 'Operasional & Pelayanan'}**
--------------------
Menguji Teks 2...
Teks: 'Berita tentang pergantian direktur utama ini tidak...'
Sentimen Hasil: **{'sentiment': 'Neutral', 'tag_new': 'Kinerja & Keuangan'}**
--------------------
Menguji Teks 3...
Teks: 'Pembatalan penerbangan tanpa pemberitahuan adalah ...'
Sentimen Hasil: **{'sentiment': 'Negative', 'tag_new': 'Operasional & Pelayanan'}**
--------------------
Menguji Teks 4...
Teks: 'Seperti Labuan Bajo, Garuda mendukung pariwisata n...'
Sentimen Hasil: **{'sentiment': 'Negative', 'tag_new': 'Industri & Pariwisata Nasional'}**
--------------------
--- Uji Coba Selesai ---


## 4. Eksekusi Analisis Sentimen pada Data Penuh

Bagian ini membaca file input CSV yang berisi data yang akan dianalisis. Kemudian, fungsi `analyze_sentiment_openai` diterapkan ke kolom teks yang ditentukan untuk setiap baris data. Hasil sentimen disimpan dalam kolom baru. Progress bar ditampilkan selama proses ini menggunakan library `tqdm`. Setelah analisis selesai, dataframe yang diperbarui disimpan ke file output CSV.

In [88]:
# ----------------------------------------------------------------------
# --- EKSEKUSI ANALISIS SENTIMEN PADA DATA PENUH ---
# Bagian ini membaca file input CSV, menerapkan fungsi analisis sentimen
# ke kolom yang ditentukan, dan menyimpan hasilnya ke file output CSV baru.
# Menggunakan tqdm untuk menampilkan progress bar selama proses analisis.
# ----------------------------------------------------------------------
from tqdm.notebook import tqdm  # tqdm versi colab (ada animasinya)
tqdm.pandas(desc="🔍 Menganalisis Sentimen") # Mengaktifkan progress bar untuk pandas apply

print(f"\nMemuat data dari: {INPUT_FILE}")

try:
    # Membaca dataset dari file CSV
    df = pd.read_csv(INPUT_FILE)
    print(f"Data dimuat. Jumlah baris: {len(df)}")

    # Memastikan kolom yang akan dianalisis ada dalam dataframe
    if COLUMN_TO_ANALYZE not in df.columns:
        raise KeyError(f"Kolom '{COLUMN_TO_ANALYZE}' tidak ditemukan di file CSV.")

    print(f"Mulai menganalisis sentimen untuk {len(df)} baris dengan model {OPENAI_MODEL}.\n")

    # Menerapkan fungsi analisis sentimen, hasilnya dictionary {sentiment, tag}
    results = df[COLUMN_TO_ANALYZE].progress_apply(analyze_sentiment_openai)

    # Pisahkan hasil dictionary jadi dua kolom
    df["sentiment"] = results.apply(lambda x: x["sentiment"])
    df["tag_new"] = results.apply(lambda x: x["tag_new"])

    print("\nAnalisis Sentimen dan Tagging Selesai! 🎉")

    # Menyimpan dataframe yang sudah ditambahkan kolom sentimen ke file CSV baru
    df.to_csv(OUTPUT_FILE, index=False)

    print(f"\nHasil Sentimen dan Tag (5 Baris Pertama):")
    # Menampilkan beberapa baris pertama dari kolom teks asli dan kolom sentimen
    print(df[[COLUMN_TO_ANALYZE, "sentiment", "tag_new"]].head())
    print("\n---------------------------------------------------------")
    print(f"✅ Data tersimpan ke file baru: {OUTPUT_FILE}")

except FileNotFoundError:
    print(f"ERROR: File '{INPUT_FILE}' tidak ditemukan. Pastikan Anda sudah mengunggahnya ke Colab.")
except KeyError as e:
    print(f"ERROR: {e}")
except Exception as e:
    print(f"Terjadi kesalahan yang tidak terduga di loop utama: {e}")


Memuat data dari: 5-garudaindonesia_news_stem.csv
Data dimuat. Jumlah baris: 469
Mulai menganalisis sentimen untuk 469 baris dengan model gpt-4.1.



🔍 Menganalisis Sentimen:   0%|          | 0/469 [00:00<?, ?it/s]


Analisis Sentimen dan Tagging Selesai! 🎉

Hasil Sentimen dan Tag (5 Baris Pertama):
                                              konten sentiment  \
0  Garuda Indonesia Kembali RUPSLB di Tengah Isu ...   Neutral   
1  Garuda Gelar RUPSLB di Tengah Isu Masuknya Dir...   Neutral   
2  JAKARTA - Ketua Komisi V DPR Lasarus mengataka...  Negative   
3  Latar Belakang\nPada pertengahan 2023, wacana ...   Neutral   
4  --\nPlt Menteri Badan Usaha Milik Negara (BUMN...   Neutral   

                   tag_new  
0       Kinerja & Keuangan  
1     Regulasi & Kebijakan  
2     Krisis & Kontroversi  
3       Kinerja & Keuangan  
4  Operasional & Pelayanan  

---------------------------------------------------------
✅ Data tersimpan ke file baru: 6-garudaindonesia_news_stem_tag_new_dan_sentiment.csv


## 5. Recheck dan Re-run Baris yang Gagal (API Error)

Bagian ini dirancang untuk menangani kemungkinan kegagalan selama analisis sentimen, terutama yang disebabkan oleh masalah koneksi atau rate limit API (ditandai dengan "API Error"). Kode ini membaca kembali file output, mengidentifikasi baris yang gagal, dan mencoba menganalisis ulang hanya baris-baris tersebut. Hasil re-analisis kemudian memperbarui file output.

In [89]:
# ----------------------------------------------------------------------
# --- RECHECK / RE-RUN BARIS YANG GAGAL (API ERROR 429) ---
# Bagian ini memeriksa hasil analisis untuk baris yang mungkin gagal
# karena masalah koneksi atau rate limit API (ditandai dengan "API Error").
# Baris yang gagal akan dianalisis ulang.
# ----------------------------------------------------------------------
print("\n🔁 Mengecek ulang baris yang gagal (API Error)...")

try:
    df = pd.read_csv(OUTPUT_FILE)

    # cari baris error
    mask_error = df["sentiment"].astype(str).str.contains("API Error", case=False, na=False)
    failed_rows = df[mask_error]

    if len(failed_rows) == 0:
        print("✅ Tidak ada baris dengan API Error. Semua data aman.")
    else:
        print(f"⚠️ Ditemukan {len(failed_rows)} baris dengan API Error. Mengulang analisis...")

        from tqdm.notebook import tqdm
        tqdm.pandas(desc="🔁 Re-analyzing failed rows")

        # re-run untuk yang error, ambil dict hasilnya
        reanalyzed = df.loc[mask_error, COLUMN_TO_ANALYZE].progress_apply(analyze_sentiment_openai)

        # pisah hasil ke dua kolom
        df.loc[mask_error, "sentiment"] = reanalyzed.apply(lambda x: x["sentiment"])
        df.loc[mask_error, "tag_new"] = reanalyzed.apply(lambda x: x["tag_new"])

        # save ulang
        df.to_csv(OUTPUT_FILE, index=False)
        print(f"✅ Re-analisis selesai. File '{OUTPUT_FILE}' telah diperbarui (termasuk kolom tag_new).")

except Exception as e:
    print(f"Terjadi kesalahan saat re-check API Error: {e}")



🔁 Mengecek ulang baris yang gagal (API Error)...
⚠️ Ditemukan 17 baris dengan API Error. Mengulang analisis...


🔁 Re-analyzing failed rows:   0%|          | 0/17 [00:00<?, ?it/s]

✅ Re-analisis selesai. File '6-garudaindonesia_news_stem_tag_new_dan_sentiment.csv' telah diperbarui (termasuk kolom tag_new).
